In [1]:
from __future__ import print_function

In [2]:
class JobQueue:
    '''
    JobQueue Object for Starcat Multiprocessing.
    
    Submit new, recieve undone and report finished jobs to and from this object.
    
    A single job is a Python dict object.
    A job looks like this:
    
    somejob = {
        'id': int,
        'name': str,
        'func': [func_name, [args]]
    }
    '''
    
    def __init__(self):
        
        self.new_jobs = []
        self.sent_jobs = []
        self.done_jobs = []
        self.made = 0
        self.new = 0
        self.sent = 0
        self.done = 0
        
    def make(self):
        blank_job = {}
        self.made += 1
        blank_job['id'] = self.made
        blank_job['name'] = None
        blank_job['func'] = [None, []]
        return blank_job
        
    def add(self, job):
        self.new_jobs.append(job)
        self.new += 1
    
    def get(self):
        if not self.new:
            print('no jobs to to')
            job = self.make()
            return job
            
        job = self.new_jobs.pop()
        self.sent_jobs.append(job)
        self.new -= 1
        self.sent += 1
        return job
        
    def finish(self, job):
        completed_job = self.sent_jobs.pop(self.sent_jobs.index(job))
        self.done_jobs.append(completed_job)
        self.sent -= 1
        self.done += 1
    
    def report(self):
        print('\n==================')
        print('  JobList Report')
        print('------------------')
        print('made  :', self.made)
        print('ready :', self.new)
        print('doing :', self.sent)
        print('done  :', self.done)
        print('==================')
        

In [3]:
def do_job(job, queue):
    print('\n==================')
    print('    Doing Job')
    print('------------------')
    print('id   :', job['id'])
    print('name :', job['name'])
    print('func :', job['func'][0])
    print('args :', job['func'][1])
    print('==================')
    
    if (job['func'][0]):
        
        job['func'][0](job['func'][1])
    
    queue.finish(job)
    print('done')

def call_back(args):
    x, y = args
    print('[call_back] call back')
    print('[call_back] args:', x, ',', y)
    print('[call_back] result', x + y)

In [4]:
jq = JobQueue()
j1 = jq.make()
jq.report()


  JobList Report
------------------
made  : 1
ready : 0
doing : 0
done  : 0


In [5]:
j1['name'] = 'callback'
j1['func'] = [call_back, [2, 4]]
j1

{'func': [<function __main__.call_back>, [2, 4]], 'id': 1, 'name': 'callback'}

In [6]:
jq.add(j1)
jq.report()


  JobList Report
------------------
made  : 1
ready : 1
doing : 0
done  : 0


In [7]:
j2 = jq.get()
j2 == j1

True

In [8]:
jq.report()


  JobList Report
------------------
made  : 1
ready : 0
doing : 1
done  : 0


In [9]:
do_job(j2, jq)


    Doing Job
------------------
id   : 1
name : callback
func : <function call_back at 0x000000000515C208>
args : [2, 4]
[call_back] call back
[call_back] args: 2 , 4
[call_back] result 6
done


In [10]:
jq.report()


  JobList Report
------------------
made  : 1
ready : 0
doing : 0
done  : 1


In [11]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

d = threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()

(daemon    ) Starting
(non-daemon) Starting
(non-daemon) Exiting
(daemon    ) Exiting


In [ ]:
threading.Thread()